In [0]:
!pip install yfinance

In [0]:
dbutils.library.restartPython()

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import re
import time
from pyspark.sql.functions import regexp_replace

In [0]:
TICKER_TABLE = "finance_catalog.db_landing.src_raw_index_keys"
TABLE_NAME = "finance_catalog.db_landing.src_raw_stock_prices"
DEFAULT_LOOKBACK_DAYS = 365
BATCH_SIZE = 100

In [0]:
tickers = (
    spark
        .table(TICKER_TABLE)
        .select(regexp_replace("ticker", "\\.", "-").alias("ticker"))
        .toPandas()["ticker"]
        .tolist()
)

In [0]:
def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

In [0]:
ticker_data = []

for batch in chunk_list(tickers[5000:], BATCH_SIZE):
    for ticker in batch:
 
        max_date_row = spark.sql(f"SELECT MAX(date) as max_date FROM {TABLE_NAME} WHERE ticker = '{ticker}' GROUP BY ticker").collect()
        max_date = max_date_row[0]['max_date'] if max_date_row else None
        start_date = max_date if max_date else datetime.now() - timedelta(days=DEFAULT_LOOKBACK_DAYS)

        df = yf.download(ticker, period='1d', start=start_date, end=datetime.now() + timedelta(days=1), auto_adjust=True)
        df = df.reset_index()
        df.columns = df.columns.droplevel(1)
        df.columns = [re.sub(r"\W+", "_", col).strip("_").lower() for col in df.columns]
        df['ticker'] = ticker
        df['id'] = ticker + df['date'].dt.strftime('%Y%m%d')
        df.set_index('id', inplace=True)
        df.reset_index(inplace=True)
        ticker_data.append(df)

        time.sleep(1)     

    if ticker_data:
        batch_df = pd.concat(ticker_data, ignore_index=True)
        ticker_data.clear()

    batch_df['volume'] = batch_df['volume'].astype(float)

    (
        spark
            .createDataFrame(batch_df)
            .orderBy(["ticker", "date"], ascending=[True, False])
            .write
            .format("delta")
            .mode("append")
            .saveAsTable(TABLE_NAME)
    )

In [0]:
# ticker_data = []

# for ticker in tickers[5000:]:
#     max_date_row = spark.sql(f"SELECT MAX(date) as max_date FROM {TABLE_NAME} WHERE ticker = '{ticker}' GROUP BY ticker").collect()
#     max_date = max_date_row[0]['max_date'] if max_date_row else None

#     if max_date:
#         start_date = max_date
#     else:
#         start_date = datetime.now() - timedelta(days=DEFAULT_LOOKBACK_DAYS)

#     df = yf.download(ticker, period='1d', start=start_date, end=datetime.now() + timedelta(days=1), auto_adjust=True)
#     df = df.reset_index()
#     df.columns = df.columns.droplevel(1)
#     df.columns = [re.sub(r"\W+", "_", col).strip("_").lower() for col in df.columns]
#     df['ticker'] = ticker
#     df['id'] = ticker + df['date'].dt.strftime('%Y%m%d')
#     df.set_index('id', inplace=True)
#     df.reset_index(inplace=True)
#     ticker_data.append(df)

# if ticker_data:
#     df = pd.concat(ticker_data, ignore_index=True)
#     df['volume'] = df['volume'].astype(float)
                            
# (
#     spark
#         .createDataFrame(df)
#         .orderBy(["ticker", "date"], ascending=[True, False])
#         .write
#         .format("delta")
#         .mode("append")
#         .saveAsTable(TABLE_NAME)
# )

In [0]:
(
    spark
        .table(TABLE_NAME)
        .dropDuplicates(subset=['ticker', 'date'])
        .orderBy(['ticker', 'date'], ascending=[True, False])
        .write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(TABLE_NAME)
)

In [0]:
# spark.table(TABLE_NAME).write.mode("overwrite").parquet(STORAGE_URL) 

In [0]:
# spark.table(TABLE_NAME).display()
# spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")